<script async src="https://www.googletagmanager.com/gtag/js?id=UA-59152712-8"></script>
<script>
  window.dataLayer = window.dataLayer || [];
  function gtag(){dataLayer.push(arguments);}
  gtag('js', new Date());

  gtag('config', 'UA-59152712-8');
</script>

# `GiRaFFE_HO` C code library: A-to-B code

## Author: Patrick Nelson

<a id='intro'></a>

**Notebook Status:** <font color=orange><b> Self-Validated </b></font>

**Validation Notes:** This code produces the expected magnetic fields for Exact Wald, Aligned Rotator, and Alfv&eacute;n Wave initial data.

# This module presents the functionality of [GiRaFFE_HO_A2B.py](../edit/GiRaFFE_HO/GiRaFFE_HO_A2B.py) and builds on it by providing C code to incorporate the generated header files into `GiRaFFE`.

## Introduction: 
This writes and documents the C code that `GiRaFFE_HO` uses to compute the magnetic fields from the vector potential. This is a relatively straightforward calculation, but requires care to be taken in the ghost zones. 

We will need to compute $B^i$ everywhere in order to evolve $\tilde{S}_i$. However, $B^i = \epsilon^{ijk} \partial_j A_k$ requires derivatives of $A_i$, so getting $B^i$ in the ghostzones (and not just on the interior) will require some finesse. A chief aspect of this will require using lower-order finite differencing in the ghost zones. To that end, we will create header files for each finite differencing order $\leq 10$, as well as upwinded- and downwinded-derivatives at 2nd order. These will let us compute the derivative at the outermost gridpoints.


<a id='toc'></a>

# Table of Contents
$$\label{toc}$$

This notebook is organized as follows

1. [Step 1](#magnetic_field): Creating the header files
1. [Step 2](#ccode): C code
    1. [Step 2.a](#a2b): Deciding which header file to use
    1. [Step 2.b](#driver): Looping over all ghostzones
1. [Step 3](#code_validation): Code Validation against original C code
1. [Step 4](#latex_pdf_output): Output this notebook to $\LaTeX$-formatted PDF file

In [1]:
# Step 0: Add NRPy's directory to the path
# https://stackoverflow.com/questions/16780014/import-file-from-parent-directory
import os,sys
nrpy_dir_path = os.path.join("..")
if nrpy_dir_path not in sys.path:
    sys.path.append(nrpy_dir_path)

import os
import cmdline_helper as cmd
outdir = "GiRaFFE_HO/GiRaFFE_Ccode_validation/A2B/"
cmd.mkdir(outdir)

<a id='magnetic_field'></a>

## Step 1: The Magnetic Field \[Back to [top](#toc)\]
$$\label{magnetic_field}$$

We start in the usual way - import the modules we need and set `DIM = 3`. We will also import the Levi-Civita symbol from `WeylScalars_Cartesian` and use it to set the Levi-Civita tensor $\epsilon^{ijk} = [ijk]/\sqrt{\gamma}$.

In [2]:
# Step 1: The A-to-B driver
from outputC import *
import grid as gri
import indexedexp as ixp
import NRPy_param_funcs as par
import finite_difference as fin

# Set spatial dimension (must be 3 for BSSN)
DIM = 3
par.set_parval_from_str("grid::DIM",DIM)

# Set the finite-differencing order to 2
par.set_parval_from_str("finite_difference::FD_CENTDERIVS_ORDER", 2)

# Register the gridfunction gammadet. This determinant will be calculated separately
gammadet = gri.register_gridfunctions("AUXEVOL","gammadet")

# Import the Levi-Civita symbol and build the corresponding tensor.
# We already have a handy function to define the Levi-Civita symbol in WeylScalars
import WeylScal4NRPy.WeylScalars_Cartesian as weyl
LeviCivitaDDD = weyl.define_LeviCivitaSymbol_rank3()
LeviCivitaUUU = ixp.zerorank3()
for i in range(DIM):
    for j in range(DIM):
        for k in range(DIM):
            LCijk = LeviCivitaDDD[i][j][k]
            #LeviCivitaDDD[i][j][k] = LCijk * sp.sqrt(gho.gammadet)
            LeviCivitaUUU[i][j][k] = LCijk / sp.sqrt(gammadet)


We will now create the structures we need: register the gridfunctions `AD` and `BU`, declare `AD_dD` for $A_{i,j}$ and zero `BU` for $B^i$. Then, we'll build the standard form of `BU` that we will use: $$B^i = \epsilon^{ijk} A_{k,j}.$$

In [3]:
# Step 1.a: A useful function
AD = ixp.register_gridfunctions_for_single_rank1("EVOL","AD")
BU = ixp.register_gridfunctions_for_single_rank1("AUXEVOL","BU")
AD_dD = ixp.declarerank2("AD_dD","nosym")
BU = ixp.zerorank1() # BU is already registered as a gridfunction, but we need to zero its values and declare it in this scope.

for i in range(DIM):
    for j in range(DIM):
        for k in range(DIM):
            BU[i] += LeviCivitaUUU[i][j][k] * AD_dD[k][j]


<a id='ccode'></a>

## Step 2: C code \[Back to [top](#toc)\]
$$\label{ccode}$$

With the header files created to actually calculate $B^i$ from $A_k$, we will write the C code to direct how those are used. We will start by including needed header files. We will then define `REAL` as type `double` (so that we can easily change it everywhere if we want to use a different precision) and the `IDX` macros that we use to access specific points in the gridfunction arrays. 

In [4]:
%%writefile $outdir/driver_AtoB.h
#include <math.h>
#include <stdio.h>
#include <stdlib.h>

#ifndef REAL
#define REAL double
#include "NGHOSTS.h" // A NRPy+-generated file, which is set based on FD_CENTDERIVS_ORDER.
#include "../CurviBoundaryConditions/gridfunction_defines.h"
#define IDX4S(g,i,j,k) \
( (i) + Nxx_plus_2NGHOSTS0 * ( (j) + Nxx_plus_2NGHOSTS1 * ( (k) + Nxx_plus_2NGHOSTS2 * (g) ) ) )
#endif


Writing GiRaFFE_HO/GiRaFFE_Ccode_validation/A2B//driver_AtoB.h


<a id='a2b'></a>

### Step 2.a: Deciding which header file to use \[Back to [top](#toc)\]
$$\label{a2b}$$

The rest of this file will consist of two functions. The first, `compute_A2B_in_ghostzones()`, will loop over the region specified by `i0min`, `i0max`, `i1min`, `i1max`, `i2min`, and `i2max`, passed to the function from the driver function. At each point, we consider the derivatives in each of the three directions that we will need to compute the curl; if we are on the edge of the grid, we will shift the stencil we use one point so that it no longer extends to outside our computational domain. Once we have all the derivatives we will need, we simply calculate the magnetic field as we otherwise would.

In [5]:
%%writefile -a $outdir/driver_AtoB.h
inline void compute_A2B_in_ghostzones(const paramstruct *restrict params,REAL *restrict in_gfs,REAL *restrict out_gfs,i0min,i0max, i1min,i1max, i2min,i2max) {
#include "set_Cparameters.h"
    for(int i2=i2min;i2<i2max;i2++) for(int i1=i1min;i1<i1max;i1++) for(int i0=i0min;i0<i0max;i0++) {
        REAL dx_Ay,dx_Az,dy_Ax,dy_Az,dz_Ax,dz_Ay;
        switch(i0) {
            case 0:
                dx_Ay = invdx0*(-3.0/2.0*in_gfs[IDX4S(AD1GF, i0,i1,i2)] + 2*in_gfs[IDX4S(AD1GF, i0+1,i1,i2)] - 1.0/2.0*in_gfs[IDX4S(AD1GF, i0+2,i1,i2)]);
                dx_Az = invdx0*(-3.0/2.0*in_gfs[IDX4S(AD2GF, i0,i1,i2)] + 2*in_gfs[IDX4S(AD2GF, i0+1,i1,i2)] - 1.0/2.0*in_gfs[IDX4S(AD2GF, i0+2,i1,i2)]);
                break;
            case Nxx_plus_2NGHOSTS0-1:
                dx_Ay = invdx0*((3.0/2.0)*in_gfs[IDX4S(AD1GF, i0,i1,i2)] - 2*in_gfs[IDX4S(AD1GF, i0-1,i1,i2)] + (1.0/2.0)*in_gfs[IDX4S(AD1GF, i0-2,i1,i2)]);
                dx_Az = invdx0*((3.0/2.0)*in_gfs[IDX4S(AD2GF, i0,i1,i2)] - 2*in_gfs[IDX4S(AD2GF, i0-1,i1,i2)] + (1.0/2.0)*in_gfs[IDX4S(AD2GF, i0-2,i1,i2)]);
                break;
            default:
                dx_Ay = invdx0*(-1.0/2.0*in_gfs[IDX4S(AD1GF, i0-1,i1,i2)] + (1.0/2.0)*in_gfs[IDX4S(AD1GF, i0+1,i1,i2)]);
                dx_Az = invdx0*(-1.0/2.0*in_gfs[IDX4S(AD2GF, i0-1,i1,i2)] + (1.0/2.0)*in_gfs[IDX4S(AD2GF, i0+1,i1,i2)]);
        }
        switch(i1) {
            case 0:
                dy_Ax = invdx1*(-3.0/2.0*in_gfs[IDX4S(AD0GF, i0,i1,i2)] + 2*in_gfs[IDX4S(AD0GF, i0,i1+1,i2)] - 1.0/2.0*in_gfs[IDX4S(AD0GF, i0,i1+2,i2)]);
                dy_Az = invdx1*(-3.0/2.0*in_gfs[IDX4S(AD2GF, i0,i1,i2)] + 2*in_gfs[IDX4S(AD2GF, i0,i1+1,i2)] - 1.0/2.0*in_gfs[IDX4S(AD2GF, i0,i1+2,i2)]);
                break;
            case Nxx_plus_2NGHOSTS1-1:
                dy_Ax = invdx1*((3.0/2.0)*in_gfs[IDX4S(AD0GF, i0,i1,i2)] - 2*in_gfs[IDX4S(AD0GF, i0,i1-1,i2)] + (1.0/2.0)*in_gfs[IDX4S(AD0GF, i0,i1-2,i2)]);
                dy_Az = invdx1*((3.0/2.0)*in_gfs[IDX4S(AD2GF, i0,i1,i2)] - 2*in_gfs[IDX4S(AD2GF, i0,i1-1,i2)] + (1.0/2.0)*in_gfs[IDX4S(AD2GF, i0,i1-2,i2)]);
                break;
            default:
                dy_Ax = invdx1*(-1.0/2.0*in_gfs[IDX4S(AD0GF, i0,i1-1,i2)] + (1.0/2.0)*in_gfs[IDX4S(AD1GF, i0,i1+1,i2)]);
                dy_Az = invdx1*(-1.0/2.0*in_gfs[IDX4S(AD2GF, i0,i1-1,i2)] + (1.0/2.0)*in_gfs[IDX4S(AD2GF, i0,i1+1,i2)]);
        }
        switch(i2) {
            case 0:
                dz_Ax = invdx2*(-3.0/2.0*in_gfs[IDX4S(AD0GF, i0,i1,i2)] + 2*in_gfs[IDX4S(AD0GF, i0,i1,i2+1)] - 1.0/2.0*in_gfs[IDX4S(AD0GF, i0,i1,i2+2)]);
                dz_Ay = invdx2*(-3.0/2.0*in_gfs[IDX4S(AD1GF, i0,i1,i2)] + 2*in_gfs[IDX4S(AD1GF, i0,i1,i2+1)] - 1.0/2.0*in_gfs[IDX4S(AD1GF, i0,i1,i2+2)]);
                break;
            case Nxx_plus_2NGHOSTS2-1:
                dz_Ax = invdx2*((3.0/2.0)*in_gfs[IDX4S(AD0GF, i0,i1,i2)] - 2*in_gfs[IDX4S(AD0GF, i0,i1,i2-1)] + (1.0/2.0)*in_gfs[IDX4S(AD0GF, i0,i1,i2-2)]);
                dz_Ay = invdx2*((3.0/2.0)*in_gfs[IDX4S(AD1GF, i0,i1,i2)] - 2*in_gfs[IDX4S(AD1GF, i0,i1,i2-1)] + (1.0/2.0)*in_gfs[IDX4S(AD1GF, i0,i1,i2-2)]);
                break;
            default:
                dz_Ax = invdx2*(-1.0/2.0*in_gfs[IDX4S(AD0GF, i0,i1,i2-1)] + (1.0/2.0)*in_gfs[IDX4S(AD0GF, i0,i1,i2+1)]);
                dz_Ay = invdx2*(-1.0/2.0*in_gfs[IDX4S(AD1GF, i0,i1,i2-1)] + (1.0/2.0)*in_gfs[IDX4S(AD1GF, i0,i1,i2+1)]);
        }
        const REAL gammadet = out_gfs[IDX4S(GAMMADETGF, i0,i1,i2)];
        out_gfs[IDX4S(BU0GF, i0,i1,i2)] = (dy_Az-dz_Ay)/gammadet;
        out_gfs[IDX4S(BU1GF, i0,i1,i2)] = (dz_Ax-dx_Az)/gammadet;
        out_gfs[IDX4S(BU2GF, i0,i1,i2)] = (dx_Ay-dy_Ax)/gammadet;
    }
}


Appending to GiRaFFE_HO/GiRaFFE_Ccode_validation/A2B//driver_AtoB.h


<a id='driver'></a>

### Step 2.b: Looping over all ghostzones \[Back to [top](#toc)\]
$$\label{driver}$$

This function is responsible for driving the logic needed to compute $B^i$ from $A_i$ whenever it is needed in the main code (typically done along with applying BCs after a timestep in the main code). We first calculate the order that we want on the interior of the grid from the parameter `NGHOSTS`. We set the boundaries `imin` and `imax` such that they define the interior of the grid. We also poison the values of $B^i$ by setting them to `NaN` everywhere: this will allow us to immediately tell if something is causing us to skip a point in the rest of the algorithm. 

With the preliminaries handled, we call `AtoB` at `ORDER` (which is being used for the rest of the derivatives throughout `GiRaFFE`). Note that `imin` and `imax` are passed such that they define the interior of the grid, and that the `FACEX` parameters are all `NUL` here. 

In [6]:
# Make sure we set the correct templates for the interior
for i in range(DIM):
    for j in range(DIM):
        for k in range(DIM):
            BU[i] += LeviCivitaUUU[i][j][k] * AD_dD[k][j]

desc="Compute the magnetic field from the vector potential everywhere"
name="driver_A_to_B"
driver_Ccode = outCfunction(
    outfile  = "returnstring", desc=desc, name=name,
    params   ="const paramstruct *restrict params,REAL *restrict in_gfs,REAL *restrict out_gfs",
    preloop  ="""
    int imin[3] = { NGHOSTS, NGHOSTS, NGHOSTS };
    int imax[3] = { Nxx_plus_2NGHOSTS0-NGHOSTS, Nxx_plus_2NGHOSTS1-NGHOSTS, Nxx_plus_2NGHOSTS2-NGHOSTS };
""",
    body     = fin.FD_outputC("returnstring",[lhrh(lhs=gri.gfaccess("out_gfs","BU0"),rhs=BU[0]),\
                                              lhrh(lhs=gri.gfaccess("out_gfs","BU1"),rhs=BU[1]),\
                                              lhrh(lhs=gri.gfaccess("out_gfs","BU2"),rhs=BU[2])],
                              params="outCverbose=False").replace("IDX4","IDX4S"),
    postloop = """
        for(int which_gz = 0; which_gz < NGHOSTS; which_gz++) {
        // After updating each face, adjust imin[] and imax[] 
        //   to reflect the newly-updated face extents.
        compute_A2B_in_ghostzones(params,in_gfs,out_gfs,imin[0]-1,imin[0], imin[1],imax[1], imin[2],imax[2]); imin[0]--;
        compute_A2B_in_ghostzones(params,in_gfs,out_gfs,imax[0],imax[0]+1, imin[1],imax[1], imin[2],imax[2]); imax[0]++;

        compute_A2B_in_ghostzones(params,in_gfs,out_gfs,imin[0],imax[0], imin[1]-1,imin[1], imin[2],imax[2]); imin[1]--;
        compute_A2B_in_ghostzones(params,in_gfs,out_gfs,imin[0],imax[0], imax[1],imax[1]+1, imin[2],imax[2]); imax[1]++;

        compute_A2B_in_ghostzones(params,in_gfs,out_gfs,imin[0],imax[0], imin[1],imax[1], imin[2]-1,imin[2]); imin[2]--;
        compute_A2B_in_ghostzones(params,in_gfs,out_gfs,imin[0],imax[0], imin[1],imax[1], imax[2],imax[2]+1); imax[2]++;
    }
""",
    loopopts="AllPoints")

with open(os.path.join(outdir,"driver_AtoB.h"),"a") as file:
    file.write(driver_Ccode)


Now, we will handle the ghost zones. We first step down the order the next lower even number. Then, for each face (taking care to match the region specified by the `imin` and `imax` parameters with the appropriate `FACEX` values), we calculate $B^i$ in the innermost ghostzone. We also decrement `imin` and increment `imax`. This serves two purposes: the next time through the loop, `AtoB` will operate on the next-outer ghost zone, and the size of each face will increase, eventually allowing us to act on each point. 

We then repeat the process for each lower (even!) order until we reach the outermost ghost zone. 

<a id='code_validation'></a>

# Step 3: Code Validation against original C code \[Back to [top](#toc)\]
$$\label{code_validation}$$

To validate the code in this tutorial we check for agreement between the files

1. that were written in this tutorial and
1. those that are stored in `GiRaFFE_HO/GiRaFFE_Ccode_library` or generated by `GiRaFFE_HO_A2B.py`


In [7]:
import difflib
import sys

# Define the directory that we wish to validate against:
valdir = "GiRaFFE_HO/GiRaFFE_Ccode_library/A2B/"

print("Printing difference between original C code and this code...")
# Open the files to compare
files_to_check = ["B_from_A_order10.h","B_from_A_order8.h","B_from_A_order6.h","B_from_A_order4.h","B_from_A_order2.h",
                  "B_from_A_order2_dirx0_dnwind.h","B_from_A_order2_dirx0_upwind.h",
                  "B_from_A_order2_dirx1_dnwind.h","B_from_A_order2_dirx1_upwind.h",
                  "B_from_A_order2_dirx2_dnwind.h","B_from_A_order2_dirx2_upwind.h",
                  "driver_AtoB.h"]

for file in files_to_check:
    print("Checking file " + file)
    with open(os.path.join(valdir+file)) as file1, open(os.path.join(outdir+file)) as file2:
        # Read the lines of each file
        file1_lines = file1.readlines()
        file2_lines = file2.readlines()
        num_diffs = 0
        for line in difflib.unified_diff(file1_lines, file2_lines, fromfile=os.path.join(valdir+file), tofile=os.path.join(outdir+file)):
            sys.stdout.writelines(line)
            num_diffs = num_diffs + 1
        if num_diffs == 0:
            print("No difference. TEST PASSED!")
        else:
            print("ERROR: Disagreement found with .py file. See differences above.")

Printing difference between original C code and this code...
Checking file B_from_A_order10.h
No difference. TEST PASSED!
Checking file B_from_A_order8.h
No difference. TEST PASSED!
Checking file B_from_A_order6.h
No difference. TEST PASSED!
Checking file B_from_A_order4.h
No difference. TEST PASSED!
Checking file B_from_A_order2.h
No difference. TEST PASSED!
Checking file B_from_A_order2_dirx0_dnwind.h
No difference. TEST PASSED!
Checking file B_from_A_order2_dirx0_upwind.h
No difference. TEST PASSED!
Checking file B_from_A_order2_dirx1_dnwind.h
No difference. TEST PASSED!
Checking file B_from_A_order2_dirx1_upwind.h
No difference. TEST PASSED!
Checking file B_from_A_order2_dirx2_dnwind.h
No difference. TEST PASSED!
Checking file B_from_A_order2_dirx2_upwind.h
No difference. TEST PASSED!
Checking file driver_AtoB.h


FileNotFoundError: [Errno 2] No such file or directory: 'GiRaFFE_HO/GiRaFFE_Ccode_library/A2B/driver_AtoB.h'

<a id='latex_pdf_output'></a>

# Step 4: Output this notebook to $\LaTeX$-formatted PDF file \[Back to [top](#toc)\]
$$\label{latex_pdf_output}$$

The following code cell converts this Jupyter notebook into a proper, clickable $\LaTeX$-formatted PDF file. After the cell is successfully run, the generated PDF may be found in the root NRPy+ tutorial directory, with filename
[Tutorial-GiRaFFE_HO_C_code_library-A2B](TTutorial-GiRaFFE_HO_C_code_library-A2B.pdf) (Note that clicking on this link may not work; you may need to open the PDF file through another means.)

In [ ]:
!jupyter nbconvert --to latex --template latex_nrpy_style.tplx --log-level='WARN' Tutorial-GiRaFFE_HO_C_code_library-A2B.ipynb
!pdflatex -interaction=batchmode Tutorial-GiRaFFE_HO_C_code_library-A2B.tex
!pdflatex -interaction=batchmode Tutorial-GiRaFFE_HO_C_code_library-A2B.tex
!pdflatex -interaction=batchmode Tutorial-GiRaFFE_HO_C_code_library-A2B.tex
!rm -f Tut*.out Tut*.aux Tut*.log